In [2]:
%pip install unidecode
%pip install pybaseball

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import warnings
warnings.filterwarnings('ignore')
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
from unidecode import unidecode 
import os
import json
import time
import pickle
import random


# Phi-2

Phi-2 is a Transformer with 2.7 billion parameters. It was trained using the same data sources as Phi-1.5, augmented with a new data source that consists of various NLP synthetic texts and filtered websites (for safety and educational value). When assessed against benchmarks testing common sense, language understanding, and logical reasoning, Phi-2 showcased a nearly state-of-the-art performance among models with less than 13 billion parameters.

The model hasn't been fine-tuned through reinforcement learning from human feedback. The intention behind crafting this open-source model is to provide the research community with a non-restricted small model to explore vital safety challenges, such as reducing toxicity, understanding societal biases, enhancing controllability, and more.

## QA Format:

You can provide the prompt as a standalone question as follows, where the model generates the text after "." . 

```
Write a detailed analogy between mathematics and a lighthouse.
```


You can provide the prompt in the Instruct/Output format as follows, where the model generates the text after "Output:".

```
Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output: Mathematics is like a lighthouse. Just as a lighthouse guides ships safely to shore, mathematics provides a guiding light in the world of numbers and logic. It helps us navigate through complex problems and find solutions. Just as a lighthouse emits a steady beam of light, mathematics provides a consistent framework for reasoning and problem-solving. It illuminates the path to understanding and helps us make sense of the world around us.
```

You can provide them in a Chat style format, where each speaker is denoted by ```{Name}:```

```
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Well, have you tried creating a study schedule and sticking to it?
Alice: Yes, I have, but it doesn't seem to help much.
Bob: Hmm, maybe you should try studying in a quiet environment, like the library.
Alice: ...
```

## Limitations of Phi-2
- Generate Inaccurate Code and Facts: The model may produce incorrect code snippets and statements. Users should treat these outputs as suggestions or starting points, not as definitive or accurate solutions.
- Limited Scope for code: Majority of Phi-2 training data is based in Python and use common packages such as "typing, math, random, collections, datetime, itertools". If the model generates Python scripts that utilize other packages or scripts in other languages, we strongly recommend users manually verify all API uses.
- Unreliable Responses to Instruction: The model has not undergone instruction fine-tuning. As a result, it may struggle or fail to adhere to intricate or nuanced instructions provided by users.
- Language Limitations: The model is primarily designed to understand standard English. Informal English, slang, or any other languages might pose challenges to its comprehension, leading to potential misinterpretations or errors in response.
- Potential Societal Biases: Phi-2 is not entirely free from societal biases despite efforts in assuring training data safety. There's a possibility it may generate content that mirrors these societal biases, particularly if prompted or instructed to do so. We urge users to be aware of this and to exercise caution and critical thinking when interpreting model outputs.
- Toxicity: Despite being trained with carefully selected data, the model can still produce harmful content if explicitly prompted or instructed to do so. We chose to release the model to help the open-source community develop the most effective ways to reduce the toxicity of a model directly after pretraining.
- Verbosity: Phi-2 being a base model often produces irrelevant or extra text and responses following its first answer to user prompts within a single turn. This is due to its training dataset being primarily textbooks, which results in textbook-like responses.


We are going to generate a split data source containing our ``train``, ``test``, and ``valuation`` sets.

Here comes the fun part!  We must examine this data to see what it contains and what we can use to encode the information we want to know.  We want to know the outcome of each pitch to the batter and use that to build a response that contains the at-bat of each player and the subsequent event or the outcome of the at-bat.  

But first we use the cache to create a dictionary of the player UUIDs to their actual names




In [4]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
players = {}
pitcher_ = {}
batter_ = {}
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    for index, row in every_pitch.iterrows(): 
        batter = str(row.batter)
        pitcher = str(row.pitcher)
        if pitcher not in players:
            name_data = playerid_reverse_lookup([row.pitcher], key_type='mlbam')
            if not name_data.empty:
                if "name_first" in name_data and 'name_last' in name_data:
                    if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                        players[pitcher] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
                    else:
                        players[pitcher] = pitcher
                        
                    pitcher_[pitcher] = players[pitcher]
                    
        if batter not in players:
            name_data = playerid_reverse_lookup([row.batter], key_type='mlbam')
            if not name_data.empty:
                if "name_first" in name_data and 'name_last' in name_data:
                    if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                        players[batter] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
                    else:
                        players[batter] = batter
                    batter_[batter] = players[batter]
                    


       
with open(f'../data/players.json', 'w', encoding='utf-8') as f:
    json.dump(players, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()


season-2003.pickle
Gathering player lookup table. This may take a moment.
season-2004.pickle
season-2005.pickle
season-2006.pickle
season-2007.pickle
season-2008.pickle
season-2009.pickle
season-2010.pickle
season-2011.pickle
season-2012.pickle


KeyboardInterrupt: 

In [11]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
players = {}
pitcher_ = {}
batter_ = {}
player_ = {}
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    for index, row in every_pitch.iterrows(): 
        batter = str(row.batter)
        pitcher = str(row.pitcher)
        if pitcher not in players:
            name_data = playerid_reverse_lookup([row.pitcher], key_type='mlbam')
            if not name_data.empty:
                if "name_first" in name_data and 'name_last' in name_data:
                    if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                        players[pitcher] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
                    else:
                        players[pitcher] = pitcher
                        
                    player_[pitcher] =  {
                        'id':str(row.pitcher),
                        'name': players[pitcher],
                        "throws":row.p_throws
                    }
                   
                    
        if batter not in players:
            name_data = playerid_reverse_lookup([row.batter], key_type='mlbam')
            if not name_data.empty:
                if "name_first" in name_data and 'name_last' in name_data:
                    if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                        players[batter] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
                    else:
                        players[batter] = batter

                    player_[batter] = {
                            'id':str(row.batter),
                            'name':players[batter],
                            "stand":row.stand
                    }
                    
with open(f'../data/players.json', 'w', encoding='utf-8') as f:
    json.dump(players, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()         

with open(f'../data/players_data.json', 'w', encoding='utf-8') as f:
    json.dump(player_, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()               

season-2003.pickle
season-2004.pickle
season-2005.pickle
season-2006.pickle
season-2007.pickle
season-2008.pickle
season-2009.pickle
season-2010.pickle
season-2011.pickle
season-2012.pickle
season-2014.pickle
season-2015.pickle
season-2016.pickle
season-2018.pickle
season-2019.pickle
season-2020.pickle
season-2021.pickle
season-2022.pickle
season-2023.pickle


In [ ]:
with open(f'../data/players.json', 'r', encoding='utf-8') as f:
    players = json.load(f)

In [6]:
from decimal import Decimal
from pandas._libs.missing import NAType

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if obj is None:
            return ""
        if isinstance(obj, Decimal):
            return str(obj)
        if isinstance(obj, NAType):
            return ""
        # 👇️ otherwise use the default behavior
        return json.JSONEncoder.default(self, obj)

In [14]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
baseball = []
input_data = []
results_data = []
lines = []
json_data = []
test = []
season_count = 0
for file in files:
    print(file)

    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    season = []
    at_bat = []
    pre_batter = ""
    for index, row in every_pitch.iterrows():
        current_batter = str(row.batter)

        if current_batter not in pre_batter:
            pre_batter = current_batter
            at_bat = []
            pitch_type = []
            release_speed = []
            pitch_names = []
            description = []


        at_bat.append({ "description":row.description,
                        "release_speeds":row.release_speed,
                        "pitch_names":row.pitch_name
                        })

        if isinstance(row.events, str):
            # on_1b = {
            #         'id':row.on_1b,
            #         "name":""
            #     }
            # if str(row.on_1b) in players:
            #     on_1b = {
            #         'id':row.on_1b,
            #         "name":players[str(row.on_1b)]
            #     }

            # on_2b = {
            #         'id':row.on_2b,
            #         "name":""
            #     }
            # if str(row.on_2b) in players:
            #     on_2b = {
            #         'id':row.on_2b,
            #         "name":players[str(row.on_2b)]
            #     }

            # on_3b = {
            #         'id':row.on_3b,
            #         "name":""
            #     }
            # if str(row.on_3b) in players:
            #     on_3b = {
            #         'id':row.on_3b,
            #         "name":players[str(row.on_3b)]
            #     }


            input = {
                        'pitcher':{
                            'id':str(row.pitcher),
                            'name':players[str(row.pitcher)] if str(row.pitcher) in players else str(row.pitcher),
                            "throws":row.p_throws
                        },
                        'batter':{
                            'id':str(row.batter),
                            'name':players[str(row.batter)] if str(row.batter) in players else str(row.batter),
                            "stand":row.stand
                        },
                        "at_bat_number":row.at_bat_number,
                        "inning_topbot":row.inning_topbot,
                        "inning":row.inning,
                        "outs_when_up":row.outs_when_up,
                        "on_1b":False if str(row.on_1b) == "<NA>" else True,
                        "on_2b":False if str(row.on_2b) == "<NA>" else True,
                        "on_3b":False if str(row.on_3b) == "<NA>" else True,
                        "home_score":row.home_score,
                        "away_score":row.away_score,
                        "pitch_number":row.pitch_number,
                    }
            
            result = {
                        "event":row.events,
                        "text":unidecode(f"{row.des}"),
                        "rbi":row.post_bat_score-row.bat_score,
                        "at_bat":at_bat,
                    }
            
            input = json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)
            output = json.dumps(result, ensure_ascii=True, allow_nan=True, cls=MyEncoder)
            json_data.append(f'''Instruct: {input}. \nOutput: {output}\n''')

                        
        season_count+=1
test = random.sample(json_data,  int(0.10*len(json_data)))
# json_data = [x for x in json_data if x not in test]   
training = json_data



output = {
    'train':{
        'text':training
    },
    'test':{
        'text':test
    }
}

with open(f'mlb_struct_encoded_2003_2023.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

season-2003.pickle
season-2004.pickle
season-2005.pickle
season-2006.pickle
season-2007.pickle
season-2008.pickle
season-2009.pickle
season-2010.pickle
season-2011.pickle
season-2012.pickle
season-2014.pickle
season-2015.pickle
season-2016.pickle
season-2018.pickle
season-2019.pickle
season-2020.pickle
season-2021.pickle
season-2022.pickle
season-2023.pickle


In [3]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
baseball = []
input_data = []
results_data = []
lines = []
json_data = []
test = []
season_count = 0
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    season = []
    at_bat = []
    pre_batter = ""
    for index, row in every_pitch.iterrows():
        if isinstance(row.events, str):
            input = {
                        'pitcher':{
                            'id':str(row.pitcher),
                            'name':players[str(row.pitcher)] if str(row.pitcher) in players else str(row.pitcher),
                            "throws":row.p_throws
                        },
                        'batter':{
                            'id':str(row.batter),
                            'name':players[str(row.batter)] if str(row.batter) in players else str(row.batter),
                            "stand":row.stand
                        },
                        "at_bat_number":row.at_bat_number,     
                        "inning_topbot":row.inning_topbot,
                        "inning":row.inning,
                        "outs_when_up":row.outs_when_up,
                        "on_1b":False if str(row.on_1b) == "<NA>" else True,
                        "on_2b":False if str(row.on_2b) == "<NA>" else True,
                        "on_3b":False if str(row.on_3b) == "<NA>" else True,
                        "home_score":row.home_score,
                        "away_score":row.away_score,
                        "pitch_number":row.pitch_number,
                    }
            
            result = {
                        "event":row.events
                    }
            
            input = json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)
            output = json.dumps(result, ensure_ascii=True, allow_nan=True, cls=MyEncoder)
            data = {
                    "text":f'''Instruct: {input}. \n Output: {output}\n''',
                    "input":f'''Instruct: {input}. \n''',
                    "output":f'''Output: {output}\n'''
                }
            json_data.append(data)

                            
            season_count+=1
        
training = random.sample(json_data, int(0.9*len(json_data)))
json_data = [x for x in json_data if x not in training]
# evaluate = random.sample(json_data,  int(0.05*len(json_data)))
# json_data = [x for x in json_data if x not in evaluate]
test = random.sample(json_data,  int(0.05*len(json_data)))

output = {
    'train':training,
    'test':test,
    'evaluate':evaluate
}

with open(f'mlb_2003_2023_phi-2_event_struct_encoded.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

season-2003.pickle


NameError: name 'players' is not defined

In [19]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
prompts = []
json_data = []
season_count = 0
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    for index, row in every_pitch.iterrows():
        if isinstance(row.events, str):
            instruction = f"what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}"
            input = f"{row.inning_topbot} of the {row.inning} inning with {row.outs_when_up} outs\n\n"
            pitch = f" a {f'{int(row.release_speed)} miles per hour ' if isinstance(row.release_speed, float) else ''}{row.pitch_name}"
            response = f"With {row.strikes} strike and {row.balls} balls, {row.pitcher} throws{pitch if isinstance(row.release_speed, float) else ''} to {row.batter}. {row.des}"
            prompts.append(f"Instruct:\n{instruction}. \nOutput:\n{response}\n")
            json_data.append({
                "instruction":instruction,
                "output":response
            })

training = random.sample(prompts, int(0.9*len(prompts)))
evaluate = random.sample(prompts,  int(0.1*len(prompts)))
test = random.sample(prompts,  int(0.1*len(prompts)))
output = {
    'train':{
        'text':training
    },
    'test':{
        'text':evaluate
    },
    'evaluate':{
        'text':test
    }
}

with open(f'../data/2011_2023_phi-2_text_encoded.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

with open(f'../data/2011_2023_phi-2_json.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

season-2011.pickle
season-2012.pickle
season-2014.pickle
season-2015.pickle
season-2016.pickle
season-2018.pickle
season-2019.pickle
season-2020.pickle
season-2021.pickle
season-2022.pickle
season-2023.pickle


In [1]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
baseball = []
input_data = []
results_data = []
lines = []
json_data = []
test = []
season_count = 0
for pitcher in pitcher_:
    for batter in batter_:
        input = {
                    'pitcher':{
                        'id':pitcher,
                        'name':players[pitcher] if pitcher in players else ""
                    },
                    'batter':{
                        'id':batter,
                        'name':players[batter] if batter in players else ""
                    },
                    "at_bat_number":random.randint(0, 100),
                    "p_throws":"R" if random.randint(0, 100) < 50 else "L",
                    "stand":"R" if random.randint(0, 100) < 50 else "L",
                    "inning_topbot":"top" if random.randint(0, 100) < 50 else "bot",
                    "inning":random.randint(1, 10),
                    "outs_when_up":random.randint(0, 3),
                    "on_1b":{
                        'id':"",
                        'name':""
                    },
                    "on_2b":{
                        'id':"",
                        'name':""
                    },
                    "on_3b":{
                        'id':"",
                        'name':""
                    },
                    "home_score":random.randint(0, 16),
                    "away_score":random.randint(0, 16),
                    "pitch_number":random.randint(0, 9),
                }
        
        input = json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)
        json_data.append(input)


output = {
    "eval":{
        "text":json_data
    }
}


with open(f'mlb_2011_2023_phi-random_data.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

NameError: name 'os' is not defined

In [ ]:
import csv
import json
def convert_csv_to_json(csv_file_path):
    # Read CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
    # Convert CSV data to JSON
    json_data = json.dumps(rows, indent=4)
    print(json_data)
    # Save JSON data to a file (optional)
    with open('../data/2018_Central_Park_Squirrel_Census_-_Stories_20240204.json', 'w') as json_file:
        json_file.write(json_data)

    return json_data

# Specify the path to your CSV file
csv_file_path = '../data/2018_Central_Park_Squirrel_Census_-_Stories_20240204.csv'

# Convert CSV to JSON
json_data = convert_csv_to_json(csv_file_path)

print("Conversion completed. JSON data:")
# print(json_data)
json_data = json.loads(json_data)
squire_stories = {}
squire_stories['train'] = {}
squire_stories['train']['text'] = []
for i in json_data:
    squire_stories['train']['text'].append(f"Question: What is an observation about a Squirrel in central park? \n Output: {unidecode(i['Note Squirrel & Park Stories'])}\n")

with open(f'../data/Squirrel.json', 'w', encoding='utf-8') as f:
    json.dump(squire_stories, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
    f.close()